In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Defino **ROI_selection**, función para poder seleccionar la región de interés (ROI) a partir de la imagen pasada como argumento

In [2]:
def ROI_selection(imgage_name):
    im = cv2.imread(imgage_name)

    # Select ROI
    r = cv2.selectROI(im)

    # Crop image
    imCrop = im[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]

    # Display cropped image
    cv2.imshow("Image", imCrop)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit(1)
    print('The coordinates for the selected ROI are: ' + str(r))
    return r, imCrop

In [6]:
ROI_coords, roi = ROI_selection("Sample_image3.jpg")

The coordinates for the selected ROI are: (314, 299, 161, 263)


In [7]:
# Puntero al video
cap = cv2.VideoCapture(0)

# Pasamos la ROI a HSV (los valores de H son más estables a cambios de intensidad)
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
# Definimos una máscara para el histograma (Hue: 0..180, Saturation:60..255, Value:32..255)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
# Y calculamos el histograma sobre esa máscara (toma solo el Hue: tonalidad)
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
# Normalizamos el histograma (entre 0 y 255 según indica el flag cv.NORM_MINMAX)
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX);

In [8]:
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 1, 10 )
track_window = ROI_coords
while(1):
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, None, fx=5, fy=5)
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        # Aplicamos camshift al nuevo frame
        #----------------------------------
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # Dibujamos la ventana sobre la imagen
        #=====================================
        
        # Con escalamiento y rotación de la ventana de seguimiento
        #---------------------------------------------------------
        pts = cv2.boxPoints(ret)                      # Encuentra los 4 vértices del rectángulo rotado
        pts = np.int0(pts)                           # int0 (entero usado para indexado)
        img2 = cv2.polylines(frame,[pts],True, 255,2) # Dibujo del rectángulo
        
        # Con escalamiento de ventana - Sin rotación
        #-------------------------------------------
#         x,y,w,h = track_window
#         img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        
        cv2.imshow('Seguimiento',img2)
        
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()

KeyboardInterrupt: 

In [19]:
cv2.destroyAllWindows()
cap.release()